In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split


tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.0-rc1


In [2]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import nltk
import json
from pathlib import Path

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences


from multiprocessing import cpu_count, Pool
import gc

In [3]:
tqdm_pandas(tqdm)

## Loading file     

In [4]:
df_train = pd.read_csv('pikabu.csv',sep=',')

In [5]:
df_train.head(20)

,context,answer
0,Убрал 600 литров мусора в реликтовом лесу. Ты...,Спасибо.
1,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Приедь к нам в Мурманск пожалуйста.
2,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Тебе платят за это?
3,Ты просто большой молодец. Спасибо. Тебе платя...,"За посты на Пикабу? Да, платят. Достаточно при..."
4,Спасибо. Тебе платят за это? За посты на Пикаб...,Не нажимается сук
5,Ты просто большой молодец. Спасибо. Тебе платя...,Лично Путин
6,Спасибо. Тебе платят за это? Лично Путин,дарт вейдар
7,Убрал 600 литров мусора в реликтовом лесу. Ты...,"ну всё, теперь можно не убирать за собой, Чист..."
8,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Зачем заминусили человека? Очевидная ирония же...
9,Убрал 600 литров мусора в реликтовом лесу.,"Герой, про которого я бы читал комиксы от dc и..."


In [14]:
df_train.count()

context    21486165
answer     21486162
dtype: int64

In [22]:
len(df_train.loc[df_train['context'].apply(len)>410,'context'][271713].split())

67

## Preprocessing

{4: 'убирать_VERB'}

In [6]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [43]:
mystem = Mystem()

In [11]:
context = df_train['context'].values

In [43]:
def index_tokenizer(tokenizer, corpus):
    #voc_set = set()
    res = np.fromiter(map(tokenizer,tqdm_notebook(corpus[:20]),),dtype=str)
    voc_set = np.unique(res)
    print(voc_set)

In [42]:
def fu(x):
    return np.apply_along_axis(text_prep_tags,1,np.reshape(x,(-1,1)))
        
def parallelize(data,partitions):

    data_split = np.array_split(data, partitions)
    pool = Pool(partitions)
    #data = np.stack(pool.apply(np.apply_along_axis,text_prep_tags,1,np.reshape(data_split,(-1,1))))
    data = np.concatenate(pool.map(fu,data_split))
    pool.close()
    pool.join()
    return data

In [33]:

if type(6) is not str:
        print('1')

1


In [31]:
re.sub(r'([ _!?])+',r'\1','gsd____gdsg   dgsdg dfdsg!     fdfs')

'gsd_gdsg dgsdg dfdsg fdfs'

In [34]:
text_id_vocab = {}
id_text_vocab = {}
prev_word = None

def text_prep_tags(text):
   
    #print(text)
    text = text[0]
    if type(text) is not str:
        text= str(text)
    upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}
    text = text.lower()
    result = np.array([])
    
    
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip() 
    text = re.sub(r'([ _!?])+',r'\1',text)
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
#     if id_text_vocab != {}:
#         prev_word = id_text_vocab[max(id_text_vocab.keys())]
#     else:       
#         prev_word = None
        
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            #result.append(token)
            result = np.append(result,token)
            
#             if prev_word:
#                 text_id_vocab[token] = text_id_vocab[prev_word] + 1
#                 id_text_vocab[text_id_vocab[prev_word] + 1] = token
#             else:
#                 text_vocab[token] = 4
#                 id_text_vocab[4] = token
#             prev_word = token

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    zer = np.empty([40], dtype="U25")
    zer[:] = '<PAD>'
    size = result.shape[0]
    if size>40:
        zer = result[:40]
    else:
        zer[:size] = result
        
    return zer

In [52]:
class Vocab:
    def __init__(self):
        
        self.char2idx = {'<PAD>':0,'<START>':1,'<END>':2,'<UNK>':3}
        self.idx2char = {i:ch for ch,i in self.char2idx.items()}
        self.last_index = 4
    
    def save(self, path='.'):
        json_ch_idx = json.dumps(self.char2idx)
        with open(os.path.join(path,"char2idx.json"),"w") as f:
            f.write(json_ch_idx)
        json_idx_ch = json.dumps(self.idx2char)
        with open(os.path.join(path,"idx2char.json"),"w") as f:
            f.write(json_idx_ch)
    
    def load(self, path='.'):       
        with open(os.path.join(path,"char2idx.json"),"r") as f:
            self.char2idx = json.loads(f.read())        
        with open(os.path.join(path,"idx2char.json"),"r") as f:
            self.idx2char = json.loads(f.read())
        
    
    def indx_tokenize_from_files(self, huge_parts, name):
        if not os.path.exists(f'{name}_indexed_token_files'):
            os.mkdir(f'{name}_indexed_token_files')
        for i in tqdm_notebook(range(huge_parts)):
            with np.load(f'{name}_tokenized_files/np_tokens{i}.npz') as data:
                np_idxs = np.apply_along_axis(np.vectorize(self.char2idx.get),1,data['arr'])
                np.savez_compressed(f'{name}_indexed_token_files/indx_tokens{i}',arr=np_idxs)
                print(f'{name} indx_tokenize part - {i}')
                
        
    
    def indx_detokenize(self, sequence):
        return ''.join([self.idx2char[idx] for idx in sequence])
    
    def tokenize(self, data, tokenizer, huge_parts, name ,save_to_file=True):
        huge_splits = np.array_split(data, huge_parts)
        partitions = cpu_count()
        for split_number,huge_split in enumerate(tqdm_notebook(huge_splits)):
                      
            data_split = np.array_split(huge_split, partitions)
            pool = Pool(partitions)

            res_data = np.concatenate(pool.map(tokenizer,data_split))
            pool.close()
            pool.join()

            self._extend_vocab(res_data)
            
            print(f'{name} first tokenize part - {split_number}')
            
            if save_to_file:
                if not os.path.exists(f'{name}_tokenized_files'):
                    os.mkdir(f'{name}_tokenized_files')
                np.savez_compressed(f'{name}_tokenized_files/np_tokens{split_number}',arr=res_data)             
                del res_data
                gc.collect()
                
                
    def _extend_vocab(self,token_matrix):
        print(f'enter {self.last_index}')
        start_idx = self.last_index
        next_indx = 0
        for idx,token in np.ndenumerate(np.unique(token_matrix)):
            if token not in self.char2idx:
                next_indx = start_idx + idx[0]
                self.char2idx[token] = next_indx
                
        if next_indx > self.last_index:
            self.last_index = next_indx+1
        print(f'exit {self.last_index}')
        self.idx2char = {i:ch for ch,i in self.char2idx.items()}
                
        
    
    def __len__(self):
        return len(self.char2idx)

In [58]:
def help_token(x):
    return np.apply_along_axis(text_prep_tags,1,np.reshape(x,(-1,1)))

voc = Vocab()


In [59]:
%time voc.tokenize(df_train['context'].values,help_token,20,name = 'context')


enter 4
exit 199505
context first tokenize part - 0
enter 199505
exit 372784
context first tokenize part - 1
enter 372784
exit 546224
context first tokenize part - 2
enter 546224
exit 721705
context first tokenize part - 3
enter 721705
exit 899093
context first tokenize part - 4
enter 899093
exit 1067692
context first tokenize part - 5
enter 1067692
exit 1238877
context first tokenize part - 6
enter 1238877
exit 1419549
context first tokenize part - 7
enter 1419549
exit 1600915
context first tokenize part - 8
enter 1600915
exit 1781883
context first tokenize part - 9
enter 1781883
exit 1960969
context first tokenize part - 10
enter 1960969
exit 2139124
context first tokenize part - 11
enter 2139124
exit 2321055
context first tokenize part - 12
enter 2321055
exit 2499659
context first tokenize part - 13
enter 2499659
exit 2685939
context first tokenize part - 14
enter 2685939
exit 2863019
context first tokenize part - 15
enter 2863019
exit 3034366
context first tokenize part - 16
enter 

In [60]:

%time voc.tokenize(df_train['answer'].values,help_token,20,name = 'answer')

enter 3545778
exit 3806923
answer first tokenize part - 0
enter 3806923
exit 4064291
answer first tokenize part - 1
enter 4064291
exit 4305576
answer first tokenize part - 2
enter 4305576
exit 4456098
answer first tokenize part - 3
enter 4456098
exit 4607016
answer first tokenize part - 4
enter 4607016
exit 4753000
answer first tokenize part - 5
enter 4753000
exit 4899703
answer first tokenize part - 6
enter 4899703
exit 5055819
answer first tokenize part - 7
enter 5055819
exit 5213058
answer first tokenize part - 8
enter 5213058
exit 5368148
answer first tokenize part - 9
enter 5368148
exit 5521843
answer first tokenize part - 10
enter 5521843
exit 5675205
answer first tokenize part - 11
enter 5675205
exit 5830724
answer first tokenize part - 12
enter 5830724
exit 5984022
answer first tokenize part - 13
enter 5984022
exit 6141680
answer first tokenize part - 14
enter 6141680
exit 6293327
answer first tokenize part - 15
enter 6293327
exit 6440368
answer first tokenize part - 16
enter 6

In [61]:
%time voc.indx_tokenize_from_files(20,name = 'context')

context indx_tokenize part - 0
context indx_tokenize part - 1
context indx_tokenize part - 2
context indx_tokenize part - 3
context indx_tokenize part - 4
context indx_tokenize part - 5
context indx_tokenize part - 6
context indx_tokenize part - 7
context indx_tokenize part - 8
context indx_tokenize part - 9
context indx_tokenize part - 10
context indx_tokenize part - 11
context indx_tokenize part - 12
context indx_tokenize part - 13
context indx_tokenize part - 14
context indx_tokenize part - 15
context indx_tokenize part - 16
context indx_tokenize part - 17
context indx_tokenize part - 18
context indx_tokenize part - 19
CPU times: user 9min 5s, sys: 22.9 s, total: 9min 28s
Wall time: 9min 30s


In [62]:
%time voc.indx_tokenize_from_files(20,name = 'answer')

answer indx_tokenize part - 0
answer indx_tokenize part - 1
answer indx_tokenize part - 2
answer indx_tokenize part - 3
answer indx_tokenize part - 4
answer indx_tokenize part - 5
answer indx_tokenize part - 6
answer indx_tokenize part - 7
answer indx_tokenize part - 8
answer indx_tokenize part - 9
answer indx_tokenize part - 10
answer indx_tokenize part - 11
answer indx_tokenize part - 12
answer indx_tokenize part - 13
answer indx_tokenize part - 14
answer indx_tokenize part - 15
answer indx_tokenize part - 16
answer indx_tokenize part - 17
answer indx_tokenize part - 18
answer indx_tokenize part - 19
CPU times: user 8min 40s, sys: 22.3 s, total: 9min 3s
Wall time: 9min 5s


In [76]:
def yy(x):
    print(x)
    return np.vectorize(voc1.char2idx.get)(x)
np.apply_along_axis(yy,1,np.array([['молодец_NOUN', 'самый_DET' ,'начало_NOUN'],
 ['это_PRON' ,'человек_NOUN', 'виноватый_ADJ'],
 ['беда_NOUN', 'гринлайт_NOUN' ,'плюсонуть_VERB']]))

['молодец_NOUN' 'самый_DET' 'начало_NOUN']
['это_PRON' 'человек_NOUN' 'виноватый_ADJ']
['беда_NOUN' 'гринлайт_NOUN' 'плюсонуть_VERB']


array([[ 99766, 149023, 105369],
       [188210, 181013,  45919],
       [ 36253,  56266, 126309]])

In [67]:
voc.save()

In [53]:
voc_test = Vocab()
%time voc_test.tokenize(df_train['context'].values[:1000000],help_token,10,name = 'test')

enter 4
exit 46395
test first tokenize part - 0
enter 46395
exit 92885
test first tokenize part - 1
enter 92885
exit 139261
test first tokenize part - 2
enter 139261
exit 185802
test first tokenize part - 3
enter 185802
exit 231676
test first tokenize part - 4
enter 231676
exit 277314
test first tokenize part - 5
enter 277314
exit 321918
test first tokenize part - 6
enter 321918
exit 367098
test first tokenize part - 7
enter 367098
exit 411912
test first tokenize part - 8
enter 411912
exit 455786
test first tokenize part - 9
CPU times: user 32.8 s, sys: 8.84 s, total: 41.6 s
Wall time: 3min 8s


In [66]:
len(voc.char2idx)

2335478

In [36]:
len(voc)

2335478

In [24]:
df_train['answer'].values.astype(str).dtype

dtype('<U140')

In [28]:
#del answer
gc.collect()
del res_data
gc.collect()

NameError: name 'res_data' is not defined

In [72]:

def read_token_files(name , number):
    np_arr = None
    for i in range(number):
        with np.load(f'{name}_tokenized_files/np_tokens{i}.npz') as data:
            if np_arr is not None:
                np_arr = np.append(np_arr,data['arr'],axis=0)
            else:
                np_arr = data['arr']
    return np_arr

In [73]:
test_context = read_token_files('context', 1)
test_answer = read_token_files('answer', 1 )

In [82]:
# del test_context
# del test_answer
gc.collect()

8

In [81]:
np_arr = None
for i in range(1):
    with np.load(f'indexed_token_files/indx_tokens{i}.npz') as data:
        if np_arr is not None:
            np_arr = np.append(np_arr,data['arr'],axis=0)
        else:
            np_arr = data['arr']

In [82]:
np_arr.shape

(1000000, 40)

In [83]:
np_arr.nbytes/1000000

320.0

array([[128897, 155086, 150350, ...,      4,      4,      4],
       [111158,   2771,  42423, ...,  11618, 115962,  42423],
       [158623, 111158,  73738, ...,      4,      4,      4],
       ...,
       [160165, 132439,  89121, ...,      4,      4,      4],
       [ 89121, 188210, 123192, ...,      4,      4,      4],
       [ 54942, 170538, 172776, ...,      4,      4,      4]])

In [63]:
%time np.unique(np_context)[100

CPU times: user 264 ms, sys: 10.4 ms, total: 274 ms
Wall time: 260 ms


'139_UNKN'

## TF model

In [ ]:
word2vec = KeyedVectors.load_word2vec_format('../../model.bin', binary=True)

In [ ]:
model_dir = './model'
vocab_size = len(voc)
sentence_size = 40
embedding_size = 300

In [9]:
def input_fn(x,  params, is_training):
    dataset = tf.data.Dataset.from_tensor_slices(x)

    if is_training:
        dataset = dataset.shuffle(buffer_size=params['buffer_size'])
        dataset = dataset.repeat(count=params['num_epochs'])

    dataset = dataset.batch(params['batch_size'])
    dataset = dataset.prefetch(buffer_size=2)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()
    #return dataset

In [68]:

def initializer(shape=None, dtype=None, partition_info=None):    
    vocab_dict = voc.char2idx
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size, embedding_size))
    num_loaded = 0
    for w, i in vocab_dict.items():
        v = None
        try:
            v = word2vec[w]
        except KeyError: # не нашли такой токен в словаре
                pass
        if v is not None :
            embedding_matrix[i] = v
            num_loaded += 1
   
    embedding_matrix = embedding_matrix.astype(np.float32)
    embedding_matrix[0] = np.zeros(300)
    return embedding_matrix

In [ ]:
def model_fn(features, labels, mode, params):  
    
    # Compute predictions.
    net = params['net']
    
    encoded_features = {}  
    
    with tf.variable_scope('encoder'):
        encoded_features['anchor'] = net(features['anchor'])
    with tf.variable_scope('encoder', reuse=True):
        encoded_features['positive'] = net(features['positive'])
    with tf.variable_scope('encoder', reuse=True):
        encoded_features['negative'] = net(features['negative'])
    
    
    #predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # Compute loss.
    loss = my_triplet_loss(encoded_features,params['margin'])
    
#     # Compute evaluation metrics.
#     accuracy = tf.metrics.accuracy(labels=labels,
#                                predictions=predicted_classes,
#                                name='acc_op')
#     f1 = tf_metrics.f1(labels=labels,
#                                predictions=predicted_classes,num_classes=3,average='micro')
    
#     metrics = {'accuracy': accuracy,'f1':f1}
#     tf.summary.scalar('accuracy', accuracy[1])
    
#     # Compute evaluation
#     if mode == tf.estimator.ModeKeys.EVAL:
#         return tf.estimator.EstimatorSpec(
#             mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)